# Lab 2 - Spark
This lab will demonstrate how to perform web server log analysis with Spark. Log data is a very large, common data source and contains a rich set of information.  It comes from many sources, such as web, file, and compute servers, application logs, user-generated content,  and can be used for monitoring servers, improving business and customer intelligence, building recommendation systems, fraud detection, and much more. This lab will show you how to use Spark on real-world text-based production logs and fully harness the power of that data.

### Apache Web Server Log file format
The log files that we use for this assignment are in the [Apache Common Log Format (CLF)](http://httpd.apache.org/docs/1.3/logs.html#common) format. The log file entries produced in CLF will look something like this:
`127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1839`
 
Each part of this log entry is described below.
* **`127.0.0.1`:** this is the IP address (or host name, if available) of the client (remote host) which made the request to the server. 
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from remote machine) is not available.
* **`-`:** the "hyphen" in the output indicates that the requested piece of information (user identity from local logon) is not available. 
* **`[01/Aug/1995:00:00:01 -0400]`:** the time that the server finished processing the request. The format is:
`[day/month/year:hour:minute:second timezone]`.
* **`"GET /images/launch-logo.gif HTTP/1.0"`:** this is the first line of the request string from the client. It consists of a three components: the request method (e.g., `GET`, `POST`, etc.), the endpoint, and the client protocol version. 
* **`200`:** this is the status code that the server sends back to the client. This information is very valuable, because it reveals whether the request resulted in a successful response (codes beginning in 2), a redirection (codes beginning in 3), an error caused by the client (codes beginning in 4), or an error in the server (codes beginning in 5). The full list of possible status codes can be found in the HTTP specification ([RFC 2616](https://www.ietf.org/rfc/rfc2616.txt) section 10).
* **`1839`:** the last entry indicates the size of the object returned to the client, not including the response headers. If no content was returned to the client, this value will be "-" (or sometimes 0).
 
Using the CLF as defined above, we create a regular expression pattern to extract the nine fields of the log line. The function returns a pair consisting of a Row object and 1. If the log line fails to match the regular expression, the function returns a pair consisting of the log line string and 0. A '-' value in the content size field is cleaned up by substituting it with 0. The function converts the log line's date string into a `Cal` object using the given `parseApacheTime` function. We, then, create the primary RDD and  we'll use in the rest of this assignment. We first load the text file and convert each line of the file into an element in an RDD. Next, we use `map(parseApacheLogLine)` to apply the parse function to each element and turn each line into a pair `Row` object. Finally, we cache the RDD in memory since we'll use it throughout this notebook. The log file is available at `data/apache/apache.log`.

In [1]:
import scala.util.matching
import org.apache.spark.rdd.RDD

case class Cal(year: Int, month: Int, day: Int, hour: Int, minute: Int, second: Int)                

case class Row(host: String, clientID: String, userID: String, dateTime: Cal, method: String, endpoint: String,
               protocol: String, responseCode: Int, contentSize: Long)                

val month_map = Map("Jan" -> 1, "Feb" -> 2, "Mar" -> 3, "Apr" -> 4, "May" -> 5, "Jun" -> 6, "Jul" -> 7, "Aug" -> 8,
                    "Sep" -> 9, "Oct" -> 10, "Nov" -> 11, "Dec" -> 12)

//------------------------------------------------
// A regular expression pattern to extract fields from the log line
val APACHE_ACCESS_LOG_PATTERN = """^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)""".r

//------------------------------------------------
def parseApacheTime(s: String): Cal = {
    return Cal(s.substring(7, 11).toInt, month_map(s.substring(3, 6)), s.substring(0, 2).toInt, 
               s.substring(12, 14).toInt, s.substring(15, 17).toInt, s.substring(18, 20).toInt)
}

//------------------------------------------------
def parseApacheLogLine(logline: String): (Either[Row, String], Int) = {
    val ret = APACHE_ACCESS_LOG_PATTERN.findAllIn(logline).matchData.toList
    if (ret.isEmpty)
        return (Right(logline), 0)

    val r = ret(0)
    val sizeField = r.group(9)

    var size: Long = 0
    if (sizeField != "-")
        size = sizeField.toLong

    return (Left(Row(r.group(1), r.group(2), r.group(3), parseApacheTime(r.group(4)), r.group(5), r.group(6),
                     r.group(7), r.group(8).toInt, size)), 1)
}

//------------------------------------------------
def parseLogs(): (RDD[(Either[Row, String], Int)], RDD[Row], RDD[String]) = {
    val fileName = "data/apache/apache.log"
    
    val parsedLogs = sc.textFile(fileName).map(parseApacheLogLine).cache()
    val accessLogs = parsedLogs.filter(x => x._2 == 1).map(x => x._1.left.get)
    val failedLogs = parsedLogs.filter(x => x._2 == 0).map(x => x._1.right.get)

    val failedLogsCount = failedLogs.count()
    
    if (failedLogsCount > 0) {
        println(s"Number of invalid logline: $failedLogs.count()")
        failedLogs.take(20).foreach(println)
    }
    
    println(s"Read $parsedLogs.count() lines, successfully parsed $accessLogs.count() lines, and failed to parse $failedLogs.count()")
    
    return (parsedLogs, accessLogs, failedLogs)
}

defined class Cal
defined class Row
month_map = Map(Nov -> 11, Jul -> 7, Mar -> 3, Jan -> 1, Oct -> 10, Dec -> 12, Feb -> 2, May -> 5, Apr -> 4, Aug -> 8, Sep -> 9, Jun -> 6)
APACHE_ACCESS_LOG_PATTERN = ^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)


parseApacheTime: (s: String)Cal
parseApacheLogLine: (logline: String)(Either[Row,String], Int)
parseLogs: ()(org.apache.spark.rdd.RDD[(Either[Row,String], Int)], org.apache.spark.rdd.RDD[Row], org.apache.spark.rdd.RDD[String])


^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)

In [2]:
val (parsedLogs, accessLogs, failedLogs) = parseLogs()

Read MapPartitionsRDD[2] at map at <console>:57.count() lines, successfully parsed MapPartitionsRDD[4] at map at <console>:58.count() lines, and failed to parse MapPartitionsRDD[6] at map at <console>:59.count()


parsedLogs = MapPartitionsRDD[2] at map at <console>:57
accessLogs = MapPartitionsRDD[4] at map at <console>:58
failedLogs = MapPartitionsRDD[6] at map at <console>:59


MapPartitionsRDD[6] at map at <console>:59

(/shuttle/missions/sts-26/images/89HC140.GIF,-4)
(/cgi-bin/imagemap/countdown69?346,293,-4)
(/shuttle/missions/sts-70/images/large/KSC-95EC-0999.JPG,-1)
(/cleve.net/commerce/sjt/smalldoom1.jpg,-1)
(/history/apollo/publications/sp-350/,-11)
(/cgi-bin/imagemap/countdown70?67,250,-1)
(/shuttle/missions/sts-33/sts-33-patch-small.gif,-107)
(/htbin/wais.pl?voice,-2)
(/history/apollo-11/images/html,-1)
(/cgi-bin/imagemap/countdown69?170,288,-3)
(/cgi-bin/imagemap/fr?126,116,-1)
(/cgi-bin/imagemap/countdown69?183,0,-1)
(/news/sci.space.news/2458,-1)
(/cgi-bin/imagemap/countdown70?63,188,-1)
(/cgi-bin/imagemap/countdown69?65,178,-1)
(/images/cdrom-1-95/img0074.jpg,-1)
(/shuttle/missions/sts-69/mission-sts,-2)
(/cgi-bin/imagemap/countdown69?335,289,-3)
(/images/STS-7.JPG,-29)
(/elv/updated.gif,-1173)
(/shuttle/missions/sts-26/movies/,-3)
(/shuttle/missions/sts-4/images/,-9)
(/shuttle/missions/sts-6/mission-sts-6.ht,-1)
(/software/winvn/userguide/winvn24.gif,-11)
(/history/apollo/apollo-13.html,-

(/htbin/imagemap/Jul95stats_r?72,100,-1)
(/cgi-bin/imagemap/countdown69?307,160,-1)
(/shuttle/missions/sts-71/sts-69-info.html,-1)
(/cgi-bin/geturlstats.pl,-83)
(/cgi-bin/imagemap/countdown69?249,156,-1)
(/cgi-bin/imagemap/countdown69?20,14,-1)
(/theta.com/goodman/,-1)
(/cgi-bin/imagemap/countdown69?50,192,-1)
(/cgi-bin/imagemap/countdown69?274,288,-2)
(/htbin/wais.pl?procurement+and+pentium,-1)
(/persons/nasa-cm/bunny.gif,-2)
(/cgi-bin/imagemap/countdown69?126,17,-1)
(/cgi-bin/imagemap/countdown69?286,283,-4)
(/htbin/wais.pl?nwl,-1)
(/software/winvn/userguide/3_3_2.htm,-8)
(/cgi-bin/imagemap/countdown70?343,287,-1)
(/shuttle/missions/sts-43/images/91HC412.GIF,-5)
(/cgi-bin/imagemap/countdown69?70,183,-4)
(/history/skylab/skylab-4.jpg,-25)
(/cgi-bin/imagemap/countdown69?334,290,-5)
(/cgi-bin/imagemap/countdown70?179,286,-2)
(/asdf,-3)
(/cgi-bin/imagemap/countdown69?292,138,-1)
(/software/winvn/faq/WINVNFAQ-III-8.html,-9)
(/history/apollo/apollo-11/images/69HC810.GIF,-43)
(/htbin/imagem

...

(/cgi-bin/imagemap/countdown69?282,42,-1)
(/cgi-bin/imagemap/countdown69?333,289,-2)
(/shuttle/missions/sts-69/test/test3.gif,-2)
(/shuttle/missions/sts-66/news/sts-66-mcc-18.txt,-1)
(/cgi-bin/imagemap/countdown69?176,17,-1)
(/just_for_fun/starttrek,-1)
(/cgi-bin/imagemap/countdown69?258,115,-2)
(/history/apollo/sa-1/images/,-29)
(/persons/astronauts/e-to-h/HammondLB.txt,-3)
(/cgi-bin/imagemap/countdown69?51,192,-2)
(/shuttle/missions/sts-35/mission-sts-35.html,-176)
(/shuttle/missions/status/r91-126,-1)
(/facilities/lc39a.html,-3190)
(/cgi-bin/imagemap/countdown70?67,56,-1)
(/shuttle/missions/status/r91-137,-1)
(/cgi-bin/imagemap/countdown69?163,293,-1)
(/cgi-bin/imagemap/countdown69?215,110,-1)
(/history/mercury/mr-3/mr-3-patch-small.gif,-724)
(/cgi-bin/imagemap/countdown69?64,158,-7)
(/cgi-bin/imagemap/countdown69?404,280,-1)
(/cgi-bin/imagemap/countdown69?187,284,-5)
(/cgi-bin/imagemap/countdown69?279,152,-1)
(/history/gemini/gemini-12.html,-2)
(/cgi-bin/imagemap/countdown69?329,27

(/history/appollo-13/apollo-13.html,-1)
(/shuttle/missions/sts-58/mission-sts-58.html,-108)
(/htbin/imagemap/Jul95stats_r?117,165,-1)
(/procurement/midrange/,-3)
(/cgi-bin/imagemap/countdown69?298,283,-2)
(/cgi-bin/imagemap/fr?68,138,-1)
(/cgi-bin/imagemap/countdown69?273,286,-1)
(/cgi-bin/imagemap/countdown69?70,214,-2)
(/cgi-bin/imagemap/countdown69?79,224,-2)
(/images/flight-deck.gif,-108)
(/history/astp/astp.html,-511)
(/cgi-bin/imagemap/countdown69?185,293,-4)
(/persons/astronauts/u-to-z/VossJE.txt,-15)
(/shuttle/missions/sts-71/images/KSC-95EC-0423.gif,-199)
(/cgi-bin/imagemap/countdown69?286,291,-4)
(/shuttle/missions/51-L/mission-51.1.html,-1)
(/cgi-bin/imagemap/countdown69?286,284,-4)
(/cgi-bin/imagemap/countdown69?66,284,-1)
(/cgi-bin/imagemap/countdown69?67,254,-5)
(/history/apollo/images/footprint-logo.gif,-7283)
(/stm-057.gif,-1)
(/htbin/wais.pl?cabin+and+atmosphere,-1)
(/shuttle/missions/sts-42/sounds/,-2)
(/cgi-bin/imagemap/countdown69?58,160,-2)
(/htbin/imagemap/Aug95st

(/shuttle/missions/sts-70/images/large/KSC-95EC-0999.JPG,-1)
(/cleve.net/commerce/sjt/smalldoom1.jpg,-1)
(/cgi-bin/imagemap/countdown70?67,250,-1)
(/htbin/wais.pl?voice,-2)
(/cgi-bin/imagemap/countdown69?170,288,-3)
(/cgi-bin/imagemap/countdown69?183,0,-1)
(/cgi-bin/imagemap/countdown70?63,188,-1)
(/images/cdrom-1-95/img0074.jpg,-1)
(/cgi-bin/imagemap/countdown69?335,289,-3)
(/elv/updated.gif,-1173)
(/shuttle/missions/sts-4/images/,-9)
(/software/winvn/userguide/winvn24.gif,-11)
(/shuttle/missions/51-d/images/,-3)
(/cgi-bin/imagemap/astrohome?271,284,-1)
(/shuttle/missions/sts-66,-1)
(/statistics/images/getstats_big.gif,-132)
(/cgi-bin/imagemap/countdown69?70,160,-3)
(/APOLLO,-1)
(/cgi-bin/imagemap/countdown69?401,285,-2)
(/cgi-bin/imagemap/countdown69?64,248,-4)
(/msfc/crew/parise.html,-1)
(/shuttle/missions/sts-71/images/KSC-95EC-0589.jpg,-95)
(/software/winvn/faq/WINVNFAQ-I-5.html,-61)
(/ksc.html/shuttle/missions/missions.html,-3)
(/shuttle/missions/sts-40/images/91HC339.GIF,-3)
(/h

(/htbin/wais.pl?srb+nozzle+joint,-1)
(/cgi-bin/imagemap/onboard?100,193,-1)
(/biomed/watqual/litcite.html,-1)
(/cgi-bin/imagemap/countdown69?61,224,-5)
(/cgi-bin/imagemap/countdown69?237,294,-1)
(/cgi-bin/imagemap/countdown69?66,256,-2)
(/images/?24,4,-1)
(/cgi-bin/imagemap/countdown69?295,287,-5)
(/htbin/wais.pl?OPM+AND+JOBS,-1)
(/thesarge.html,-3)
(/shuttle/missions/news/1992/h09.08.92,-1)
(/cgi-bin/imagemap/countdown70?72,152,-1)
(/history/apollo/apollo-4/images/apollo-4.jpg,-26)
(/cgi-bin/imagemap/countdown69?56,291,-1)
(/msfc/description/instruments/uit_line_bay-small.gif,-2)
(/shuttle/countdown/video/b-shaw.jpeg,-1)
(/cgi-bin/imagemap/countdown69?56,196,-1)
(/shuttle/countdown/lps/hsp/hsp.html,-32)
(/%5B.pub.win3.winvn,-1)
(/kcs.html,-7)
(/cgi-bin/imagemap/fr?203,42,-1)
(/cgi-bin/imagemap/countdown69?53,155,-1)
(/cgi-bin/imagemap/countdown69?284,290,-5)
(/cgi-bin/imagemap/fr?147,27,-1)
(/shuttle/missions/51-f/movies/,-4)
(/cgi-bin/imagemap/countdown69?60,232,-1)
(/shuttle/mission

(/cgi-bin/imagemap/astrohome?123,203,-1)
(/htbin/wais.pl?omi,-1)
(/facts/faq13.txt,-2)
(/cgi-bin/imagemap/countdown70?56,219,-1)
(/history/appolo,-1)
(/cgi-bin/imagemap/countdown69?238,109,-1)
(/shuttle/missions/status/r93-134,-1)
(/htbin/wais.pl?mission+patches,-1)
(/cgi-bin/imagemap/countdown69?73,159,-5)
(/htbin/wais.pl?ROMPS,-6)
(/cgi-bin/imagemap/countdown69?59,155,-6)
(/shuttle.html,-1)
(/shuttle/missions/sts-26/images/89HC146.GIF,-5)
(/shuttle/missions/sts-73.html,-1)
(/cgi-bin/imagemap/countdown69?288,47,-1)
(/htbin/wais.pl?Gifts,-1)
(/mdss/srqa/homepage.txt,-1)
(/persons/astronauts/a-to-d/BoldenCF.txt,-86)
(/htbin/imagemap/Jul95stats_b?446,110,-1)
(/biomed/watqual/gif/hydrolabsmall.gif,-4)
(/winvn/winvn.html.,-1)
(/history.history.htmi,-6)
(/statistics/1993/1993.html,-13)
(/news/nasa.nasamail.p/79,-1)
(/htbin/wais.pl?Mass+Driver,-1)
(/cgi-bin/imagemap/countdown69?73,218,-5)
(/history/apollo/sa-9/sa-9-info.html,-13)
(/cgi-bin/imagemap/countdown69?190,290,-2)
(/cgi-bin/imagemap/

### Sample Analyses on the Web Server Log File
Let's compute some statistics about the sizes of content being returned by the web server. In particular, we'd like to know what are the average, minimum, and maximum content sizes. We can compute the statistics by applying a `map` to the `accessLogs` RDD. The given function to the `map` should extract the `contentSize` field from the RDD. The `map` produces a new RDD, called `contentSizes`, containing only the `contentSizes`. To compute the minimum and maximum statistics, we can use `min()` and `max()` functions on the new RDD. We can compute the average statistic by using the `reduce` function with a function that sums the two inputs, which represent two elements from the new RDD that are being reduced together. The result of the `reduce()` is the total content size from the log and it is to be divided by the number of requests as determined using the `count()` function on the new RDD. As the result of executing the following box, you should get the below result:
```
Content Size Avg: 17531, Min: 0, Max: 3421948
```

In [27]:
// TODO: Replace <FILL IN> with appropriate code
// Calculate statistics based on the content size.
val contentSizes = accessLogs.map(x => x.contentSize).cache()

val avg = contentSizes.reduce((x, y) => x+y) / contentSizes.count()

println("Content Size Avg: " + avg + ", Min: " + contentSizes.min() + ", Max: " + contentSizes.max())

Content Size Avg: 17531, Min: 0, Max: 3421948


contentSizes = MapPartitionsRDD[9] at map at <console>:33
avg = 17531


17531

Next, lets look at the "response codes" that appear in the log. As with the content size analysis, first we create a new RDD that contains the `responseCode` field from the `accessLogs` RDD. The difference here is that we will use a *pair tuple* instead of just the field itself (i.e., (response code, 1)). Using a pair tuple consisting of the response code and 1 will let us count the number of of records with a particular response code. Using the new RDD `responseCodes`, we perform a `reduceByKey` function that applys a given function to each element, pairwise with the same key. Then, we cache the resulting RDD and create a list by using the `take` function. Once you run the code below, you should receive the following results:
```
Found 7 response codes
Response Code Counts: (404,6185) (200,940847) (304,79824) (500,2) (501,17) (302,16244) (403,58)
```

In [28]:
// TODO: Replace <FILL IN> with appropriate code

// extract the response code for each record and make pair of (response code, 1)
val responseCodes = accessLogs.map(x => (x.responseCode, 1))

// count the number of records for each key
val responseCodesCount = responseCodes.reduceByKey((x, y) => x + y).cache()

// take the first 100 records
val responseCodesCountList = responseCodesCount.take(100)

println("Found " + responseCodesCountList.length + " response codes")
print("Response Code Counts: ")
responseCodesCountList.foreach(x => print(x + " "))

Found 7 response codes
Response Code Counts: (404,6185) (200,940847) (304,79824) (500,2) (501,17) (302,16244) (403,58) 

responseCodes = MapPartitionsRDD[10] at map at <console>:32
responseCodesCount = ShuffledRDD[11] at reduceByKey at <console>:35
responseCodesCountList = Array((404,6185), (200,940847), (304,79824), (500,2), (501,17), (302,16244), (403,58))


[(404,6185), (200,940847), (304,79824), (500,2), (501,17), (302,16244), (403,58)]

Let's look at "hosts" that have accessed the server multiple times (e.g., more than 10 times). First we create a new RDD to keep the `host` field from the `accessLogs` RDD using a pair tuple consisting of the host and 1 (i.e., (host, 1)), which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` function with a given function to add the two values. We then filter the result based on the count of accesses by each host (the second element of each pair) being greater than 10. Next, we extract the host name by performing a `map` to return the first element of each pair. Finally, we extract 20 elements from the resulting RDD. The result should be as below:
```
Any 20 hosts that have accessed more then 10 times: 
ix-aug-ga1-13.ix.netcom.com 
n1043347.ksc.nasa.gov 
d02.as1.nisiq.net 
192.112.22.82 
anx3p4.trib.com 
198.215.127.2 
198.77.113.34 
crc182.cac.washington.edu 
telford-107.salford.ac.uk 
universe6.barint.on.ca 
gatekeeper.homecare.com 
157.208.11.7 
unknown.edsa.co.za 
onyx.southwind.net 
ppp-hck-2-12.ios.com 
ix-lv5-04.ix.netcom.com 
f-umbc7.umbc.edu 
cs006p09.nam.micron.net 
dd22-025.compuserve.com 
hak-lin-kim.utm.edu 
```

In [34]:
// TODO: Replace <FILL IN> with appropriate code

// extract the host field for each record and make pair of (host, 1)
val hosts = accessLogs.map(x => (x.host, 1))

// count the number of records for each key
val hostsCount = hosts.reduceByKey((x, y) => x+ y)

// keep the records with the count greater than 10
val hostMoreThan10 = hostsCount.filter( x => x._2 > 10 )

// take the first 100 records
val hostsPick20 = hostMoreThan10.map( x => x._1 ).take(20)

println("Any 20 hosts that have accessed more then 10 times: ")
hostsPick20.foreach(println)

Any 20 hosts that have accessed more then 10 times: 
ix-aug-ga1-13.ix.netcom.com
n1043347.ksc.nasa.gov
d02.as1.nisiq.net
192.112.22.82
anx3p4.trib.com
198.215.127.2
198.77.113.34
crc182.cac.washington.edu
telford-107.salford.ac.uk
universe6.barint.on.ca
gatekeeper.homecare.com
157.208.11.7
unknown.edsa.co.za
onyx.southwind.net
ppp-hck-2-12.ios.com
ix-lv5-04.ix.netcom.com
f-umbc7.umbc.edu
cs006p09.nam.micron.net
dd22-025.compuserve.com
hak-lin-kim.utm.edu


hosts = MapPartitionsRDD[12] at map at <console>:32
hostsCount = ShuffledRDD[13] at reduceByKey at <console>:35
hostMoreThan10 = MapPartitionsRDD[14] at filter at <console>:38
hostsPick20 = Array(ix-aug-ga1-13.ix.netcom.com, n1043347.ksc.nasa.gov, d02.as1.nisiq.net, 192.112.22.82, anx3p4.trib.com, 198.215.127.2, 198.77.113.34, crc182.cac.washington.edu, telford-107.salford.ac.uk, universe6.barint.on.ca, gatekeeper.homecare.com, 157.208.11.7, unknown.edsa.co.za, onyx.southwind.net, ppp-hck-2-12.ios.com, ix-lv5-04.ix.netcom.com, f-umbc7.umbc.edu, cs006p09.nam.micron.net, dd22-025.compuserve.com, hak-lin-kim.utm.edu)


[ix-aug-ga1-13.ix.netcom.com, n1043347.ksc.nasa.gov, d02.as1.nisiq.net, 192.112.22.82, anx3p4.trib.com, 198.215.127.2, 198.77.113.34, crc182.cac.washington.edu, telford-107.salford.ac.uk, universe6.barint.on.ca, gatekeeper.homecare.com, 157.208.11.7, unknown.edsa.co.za, onyx.southwind.net, ppp-hck-2-12.ios.com, ix-lv5-04.ix.netcom.com, f-umbc7.umbc.edu, cs006p09.nam.micron.net, dd22-025.compuserve.com, hak-lin-kim.utm.edu]

For the final example, we'll look at the top endpoints (URIs) in the log. To determine them, we first create a new RDD to extract the `endpoint` field from the `accessLogs` RDD using a pair tuple consisting of the endpoint and 1 (i.e., (endpoint, 1)), which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` to add the two values. We then extract the top 10 endpoints by performing a `takeOrdered` with a value of 10 and a function that multiplies the count (the second element of each pair) by -1 to create a sorted list with the top endpoints at the bottom. Here is the result:
```
Top ten endpoints: 
(/images/NASA-logosmall.gif,59737)
(/images/KSC-logosmall.gif,50452)
(/images/MOSAIC-logosmall.gif,43890)
(/images/USA-logosmall.gif,43664)
(/images/WORLD-logosmall.gif,43277)
(/images/ksclogo-medium.gif,41336)
(/ksc.html,28582)
(/history/apollo/images/apollo-logo1.gif,26778)
(/images/launch-logo.gif,24755)
(/,20292)
```

In [71]:
// TODO: Replace <FILL IN> with appropriate code

// extract the endpoint for each record and make pair of (endpoiint, 1)
val endpoints = accessLogs.map(x => (x.endpoint, 1))

// count the number of records for each key
val endpointCounts = endpoints.reduceByKey((x, y) => x+ y)

// extract the top 10 
val topEndpoints = endpointCounts.map(x => (x._2 *(-1), x._1)).takeOrdered(10).map(x => (x._2, x._1 * (-1)))

println("Top ten endpoints: ")
topEndpoints.foreach(println)

Top ten endpoints: 
(/images/NASA-logosmall.gif,59737)
(/images/KSC-logosmall.gif,50452)
(/images/MOSAIC-logosmall.gif,43890)
(/images/USA-logosmall.gif,43664)
(/images/WORLD-logosmall.gif,43277)
(/images/ksclogo-medium.gif,41336)
(/ksc.html,28582)
(/history/apollo/images/apollo-logo1.gif,26778)
(/images/launch-logo.gif,24755)
(/,20292)


endpoints = MapPartitionsRDD[89] at map at <console>:37
endpointCounts = ShuffledRDD[90] at reduceByKey at <console>:40
topEndpoints = Array((/images/NASA-logosmall.gif,59737), (/images/KSC-logosmall.gif,50452), (/images/MOSAIC-logosmall.gif,43890), (/images/USA-logosmall.gif,43664), (/images/WORLD-logosmall.gif,43277), (/images/ksclogo-medium.gif,41336), (/ksc.html,28582), (/history/apollo/images/apollo-logo1.gif,26778), (/images/launch-logo.gif,24755), (/,20292))


[(/images/NASA-logosmall.gif,59737), (/images/KSC-logosmall.gif,50452), (/images/MOSAIC-logosmall.gif,43890), (/images/USA-logosmall.gif,43664), (/images/WORLD-logosmall.gif,43277), (/images/ksclogo-medium.gif,41336), (/ksc.html,28582), (/history/apollo/images/apollo-logo1.gif,26778), (/images/launch-logo.gif,24755), (/,20292)]

### Analyzing Web Server Log File
What are the top ten endpoints which did not have return code 200? Create a sorted list containing top ten endpoints and the number of times that they were accessed with non-200 return code. Think about the steps that you need to perform to determine which endpoints did not have a 200 return code, how you will uniquely count those endpoints, and sort the list. You should receive the following result:
```
Top ten failed URLs: 
(/images/NASA-logosmall.gif,8761)
(/images/KSC-logosmall.gif,7236)
(/images/MOSAIC-logosmall.gif,5197)
(/images/USA-logosmall.gif,5157)
(/images/WORLD-logosmall.gif,5020)
(/images/ksclogo-medium.gif,4728)
(/history/apollo/images/apollo-logo1.gif,2907)
(/images/launch-logo.gif,2811)
(/,2199)
(/images/ksclogosmall.gif,1622)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

// keep the logs with error code not 200
val not200 = accessLogs.filter(<FILL IN>)

// make a pair of (x, 1)
val endpointCountPairTuple = not200.map(<FILL IN>)

// count the number of records for each key x
val endpointSum = endpointCountPairTuple.reduceByKey(<FILL IN>)

// take the top 10
val topTenErrURLs = endpointSum.takeOrdered(10)(<FILL IN>)

println("Top ten failed URLs: ")
topTenErrURLs.foreach(println)

Let's count the number of unique hosts in the entire log. Think about the steps that you need to perform to count the number of different hosts in the log. The result should be as below:
```
Unique hosts: 54507

```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

// extract the host field for each record
val hosts = accessLogs.map(<FILL IN>)

// keep the uniqe hosts
val uniqueHosts = hosts.<FILL IN>

// count them
val uniqueHostCount = uniqueHosts.count()

println("Unique hosts: " + uniqueHostCount)

For an advanced exercise, let's determine the number of unique hosts in the entire log on a day-by-day basis. This computation will give us counts of the number of unique daily hosts. We'd like a list sorted by increasing day of the month, which includes the day of the month and the associated number of unique hosts for that day. Make sure you cache the resulting RDD `dailyHosts`, so that we can reuse it in the next exercise. Think about the steps that you need to perform to count the number of different hosts that make requests *each* day. Since the log only covers a single month, you can ignore the month. Here is the output you should receive:
```
Unique hosts per day: 
(1,2582)
(3,3222)
(4,4190)
(5,2502)
(6,2537)
(7,4106)
(8,4406)
(9,4317)
(10,4523)
(11,4346)
(12,2864)
(13,2650)
(14,4454)
(15,4214)
(16,4340)
(17,4385)
(18,4168)
(19,2550)
(20,2560)
(21,4134)
(22,4456)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

// make pairs of (day, host)
val dayToHostPairTuple = accessLogs.map(<FILL IN>)

// group by day
val dayGroupedHosts = dayToHostPairTuple.<FILL IN>

// make pairs of (day, number of host in that day)
val dayHostCount = dayGroupedHosts.map(<FILL IN>)

// sort by day
val dailyHosts = dayHostCount.<FILL IN>

// return the records as a list
val dailyHostsList = dailyHosts.take(30)

println("Unique hosts per day: ")
dailyHostsList.foreach(println)

Next, let's determine the average number of requests on a day-by-day basis. We'd like a list by increasing day of the month and the associated average number of requests per host for that day. Make sure you cache the resulting RDD `avgDailyReqPerHost` so that we can reuse it in the next exercise. To compute the average number of requests per host, get the total number of request across all hosts and divide that by the number of unique hosts. Since the log only covers a single month, you can skip checking for the month. Also to keep it simple, when calculating the approximate average use the integer value. The result should be as below:
```
Average number of daily requests per Hosts is: 
(1,13)
(3,12)
(4,14)
(5,12)
(6,12)
(7,13)
(8,13)
(9,14)
(10,13)
(11,14)
(12,13)
(13,13)
(14,13)
(15,13)
(16,13)
(17,13)
(18,13)
(19,12)
(20,12)
(21,13)
(22,12)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

// make pairs of (day, host)
val dayAndHostTuple = accessLogs.map(<FILL IN>)

// group by day
val groupedByDay = dayAndHostTuple.<FILL IN>

// sort by day
val sortedByDay = groupedByDay.<FILL IN>

// calculate the average request per day
val avgDailyReqPerHost = sortedByDay.map(<FILL IN>

// return the records as a list
val avgDailyReqPerHostList = avgDailyReqPerHost.take(30)

println("Average number of daily requests per Hosts is: ")
avgDailyReqPerHostList.foreach(println)

### Exploring 404 Response Codes
Let's count the 404 response codes. Create a RDD containing only log records with a 404 response code. Make sure you `cache()` the RDD `badRecords` as we will use it in the rest of this exercise. How many 404 records are in the log? Here is the result:
```
Found 6185 404 URLs.
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val badRecords = accessLogs.<FILL IN>

println("Found " + badRecords.count() + " 404 URLs.")

Now, let's list the 404 response code records. Using the RDD containing only log records with a 404 response code that you cached in the previous part, print out a list up to 10 distinct endpoints that generate 404 errors -  no endpoint should appear more than once in your list. You should receive the follwoing output as your result:
```
404 URLS: 
/SHUTTLE/COUNTDOWN
/shuttle/missions/sts-71/images/www.acm.uiuc.edu/rml/Gifs
/shuttle/technology/stsnewsrof/stsref-toc.html
/de/systems.html
/ksc.htnl
/~pccomp/graphics/sinsght.gif
/PERSONS/NASA-CM.
/shuttle/missions/sts-1/sts-1-mission.html
/history/apollo/sa-1/sa-1-patch-small.gif
/images/sts-63-Imax
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val badEndpoints = badRecords.<FILL IN>
val badUniqueEndpoints = badEndpoints.<FILL IN>
val badUniqueEndpointsPick10 = badUniqueEndpoints.take(10)

println("404 URLS: ")
badUniqueEndpointsPick10.foreach(println)

Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top 10 endpoints that generate the most 404 errors. Remember, top endpoints should be in sorted order. The result would be as below:
```
Top ten 404 URLs: 
(/pub/winvn/readme.txt,633)
(/pub/winvn/release.txt,494)
(/shuttle/missions/STS-69/mission-STS-69.html,431)
(/images/nasa-logo.gif,319)
(/elv/DELTA/uncons.htm,178)
(/shuttle/missions/sts-68/ksc-upclose.gif,156)
(/history/apollo/sa-1/sa-1-patch-small.gif,146)
(/images/crawlerway-logo.gif,120)
(/://spacelink.msfc.nasa.gov,117)
(/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif,100)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val badEndpointsCountPairTuple = badRecords.map(<FILL IN>)
val badEndpointsSum = badEndpointsCountPairTuple.reduceByKey(<FILL IN>)

// sort the records (you should implement a new object by extending the Ordering class for this part)
object ErrOrdering extends Ordering[(String, Int)] {
  <FILL IN>
}

val badEndpointsTop10 = badEndpointsSum.takeOrdered(10)(ErrOrdering.reverse)

println("Top ten 404 URLs: ")
badEndpointsTop10.foreach(println)

Instead of looking at the endpoints that generated 404 errors, now let's look at the hosts that encountered 404 errors. Using the RDD containing only log records with a 404 response code that you cached before, print out a list of the top 10 hosts that generate the most 404 errors. Here is the result:
```
Top ten hosts that generated errors: 
(piweba3y.prodigy.com,39)
(maz3.maz.net,39)
(gate.barr.com,38)
(m38-370-9.mit.edu,37)
(ts8-1.westwood.ts.ucla.edu,37)
(nexus.mlckew.edu.au,37)
(204.62.245.32,33)
(spica.sci.isas.ac.jp,27)
(163.206.104.34,27)
(www-d4.proxy.aol.com,26)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val errHostsCountPairTuple = badRecords.map(<FILL IN>)
val errHostsSum = errHostsCountPairTuple.reduceByKey(<FILL IN>)
val errHostsTop10 = errHostsSum.takeOrdered(10)(<FILL IN>)

println("Top ten hosts that generated errors: ")
errHostsTop10.foreach(println)

Let's explore the 404 records temporally. Break down the 404 requests by day and get the daily counts sorted by day as a list. Since the log only covers a single month, you can ignore the month in your checks. Cache the `errDateSorted` at the end. The output should be as below:
```
404 errors by day: 
(1,243)
(3,303)
(4,346)
(5,234)
(6,372)
(7,532)
(8,381)
(9,279)
(10,314)
(11,263)
(12,195)
(13,216)
(14,287)
(15,326)
(16,258)
(17,269)
(18,255)
(19,207)
(20,312)
(21,305)
(22,288)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val errDateCountPairTuple = badRecords.map(<FILL IN>)
val errDateSum = errDateCountPairTuple.reduceByKey(<FILL IN>)
val errDateSorted = errDateSum
val errByDate = errDateSorted.takeOrdered(30)
errDateSorted.cache()

println("404 errors by day: ")
errByDate.foreach(println)

Using the RDD `errDateSorted` you cached before, what are the top five days for 404 response codes and the corresponding counts of 404 response codes?
```
Top five dates for 404 requests: (7,532) (8,381) (6,372) (4,346) (15,326) 
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

object DateOrdering extends Ordering[(Int, Int)] {
  <FILL IN>
}

val topErrDate = errDateSorted.takeOrdered(5)(DateOrdering.reverse)

print("Top five dates for 404 requests: ")
topErrDate.foreach(x => print(x + " "))

Using the RDD `badRecords` you cached before, and by hour of the day and in increasing order, create an RDD containing how many requests had a 404 return code for each hour of the day (midnight starts at 0). 
```
Top hours for 404 requests: 
(0,175)
(1,171)
(2,422)
(3,272)
(4,102)
(5,95)
(6,93)
(7,122)
(8,199)
(9,185)
(10,329)
(11,263)
(12,438)
(13,397)
(14,318)
(15,347)
(16,373)
(17,330)
(18,268)
(19,269)
(20,270)
(21,241)
(22,234)
(23,272)
```

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val hourCountPairTuple = badRecords.map(<FILL IN>)
val hourRecordsSum = hourCountPairTuple.reduceByKey(<FILL IN>)
val hourRecordsSorted = hourRecordsSum.<FILL IN>
val errHourList = hourRecordsSorted.<FILL IN>

println("Top hours for 404 requests: ")
errHourList.foreach(println)